In [1]:
%load_ext autoreload
%load_ext autotime

time: 199 µs (started: 2024-10-09 15:42:52 -07:00)


In [2]:
import os
import sys
import time
import warnings

# Add WisIO to the path
sys.path.append(os.path.abspath("../"))

# Filter warnings
warnings.filterwarnings('ignore')

time: 591 µs (started: 2024-10-09 15:42:52 -07:00)


In [3]:
analyzer = "dftracer"
checkpoint = True
logical_view_types = False
output_max_bottlenecks = 1
output_root_only = False
run_dir = f".wisio/{analyzer}/{int(time.time())}"
bottleneck_dir = f"{run_dir}/bottlenecks"
checkpoint_dir = f"{run_dir}/checkpoints"
percentile = 0.95
trace_path = "/p/lustre3/iopp/dftracer-traces-lfs/v1.0.5-develop/corona/cm1/APP/node-1/v1/RAW/*.pfw.gz"

time: 750 µs (started: 2024-10-09 15:42:52 -07:00)


In [4]:
# Parameters
analyzer = "dftracer"
checkpoint = True
checkpoint_dir = "/usr/workspace/iopp/wisio_logs/_checkpoints/dftracer-cm1-node-1"
logical_view_types = False
output_max_bottlenecks = 1
output_root_only = False
run_dir = ".wisio/dftracer/cm1-node-1"
percentile = 0.99
trace_path = "/p/lustre3/iopp/dftracer-traces-lfs/v1.0.5-develop/corona/cm1/APP/node-1/v1/COMPACT/*.pfw.gz"


time: 655 µs (started: 2024-10-09 15:42:52 -07:00)


In [5]:
from distributed import Client
from hydra import compose, initialize
from hydra.core.hydra_config import HydraConfig
from hydra.utils import instantiate
from wisio.config import init_hydra_config_store
from wisio.utils.file_utils import ensure_dir

ensure_dir(run_dir)

with initialize(version_base=None, config_path=None):
    init_hydra_config_store()
    cfg = compose(
        config_name="config",
        overrides=[
            f"+analyzer={analyzer}",
            f"analyzer.bottleneck_dir={bottleneck_dir}",
            f"analyzer.checkpoint={checkpoint}",
            f"analyzer.checkpoint_dir={checkpoint_dir}",
            f"hydra.run.dir={run_dir}",
            f"hydra.runtime.output_dir={run_dir}",
            f"logical_view_types={logical_view_types}",
            f"output.max_bottlenecks={output_max_bottlenecks}",
            f"output.root_only={output_root_only}",
            f"percentile={percentile}",
            f"trace_path={trace_path}",
        ],
        return_hydra_config=True,
    )
    HydraConfig.instance().set_config(cfg)
    cluster = instantiate(cfg.cluster)
    client = Client(cluster)
    analyzer = instantiate(cfg.analyzer)
    output = instantiate(cfg.output)

time: 6.82 s (started: 2024-10-09 15:42:52 -07:00)


In [6]:
result = analyzer.analyze_trace(
    trace_path=cfg.trace_path,
    # accuracy=cfg.accuracy,
    exclude_bottlenecks=cfg.exclude_bottlenecks,
    exclude_characteristics=cfg.exclude_characteristics,
    logical_view_types=cfg.logical_view_types,
    metrics=cfg.metrics,
    percentile=cfg.percentile,
    view_types=cfg.view_types,
)

time: 34 s (started: 2024-10-09 15:42:59 -07:00)


In [7]:
output.handle_result(metrics=cfg.metrics, result=result)

╭────────────────────────────────────────────── I/O Characteristics ──────────────────────────────────────────────╮
│                                                                                                                 │
│  Runtime          145.36 seconds                                                                                │
│  I/O Time         1.35 seconds                                                                                  │
│                   ├── Read - 0.03 seconds (1.96%)                                                               │
│                   ├── Write - 0.15 seconds (10.86%)                                                             │
│                   └── Metadata - 1.19 seconds (88.42%)                                                          │
│  I/O Operations   231,337 ops                                                                                   │
│                   ├── Read - 13,489 ops (5.83%)                                                                 │
│                   ├── Write - 112,353 ops (48.57%)                                                              │
│                   └── Metadata - 105,063 ops (45.42%)                                                           │
│  I/O Size         690.06 MiB                                                                                    │
│                   ├── Read - 1.76 MiB (0.26%)                                                                   │
│                   └── Write - 688.30 MiB (99.74%)                                                               │
│  Read Requests    4 kiB-16 kiB - 13,489 ops                                                                     │
│                   ├── <4 kiB - 13,440 ops (99.64%)                                                              │
│                   └── 4-16 kiB - 49 ops (0.36%)                                                                 │
│  Write Requests   4 kiB-64 kiB - 112,353 ops                                                                    │
│                   ├── <4 kiB - 1,945 ops (1.73%)                                                                │
│                   ├── 4-16 kiB - 107,192 ops (95.41%)                                                           │
│                   └── 16-64 kiB - 3,216 ops (2.86%)                                                             │
│  Nodes            1 node                                                                                        │
│  Apps             1 app                                                                                         │
│  Processes/Ranks  48 processes                                                                                  │
│  Files            2,556 files                                                                                   │
│                   ├── Shared: 669 files (26.17%)                                                                │
│                   └── FPP: 1,887 files (73.83%)                                                                 │
│  Time Periods     126 time periods (Time Granularity: 1,000,000.0)                                              │
│  Access Pattern   Sequential: 125,842 ops (100.00%) - Random: 0 ops (0.00%)                                     │
│                                                                                                                 │
╰─ R: Read - W: Write - M: Metadata  ─────────────────────────────────────────────────────────────────────────────╯
                                                                                                                   
╭────────────────────────────────────── 49 I/O Bottlenecks with 147 Reasons ──────────────────────────────────────╮
│                                                                                                                 │
│  Time View (2 bottlenecks with 6 reasons)             

time: 1.99 s (started: 2024-10-09 15:43:33 -07:00)
